In [83]:
import os
from datetime import datetime
import zipfile
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import time
from sklearn.metrics import classification_report, accuracy_score
from sklearn import preprocessing


In [84]:
path=r"C:\Users\Sumit Kumar Yadav\Desktop\raw"
train1=os.path.join(path,"train/phone/gyro")
train2=os.path.join(path,"train/phone/accel")
train3=os.path.join(path,"train/watch/gyro")
train4=os.path.join(path,"train/watch/accel")

test1=os.path.join(path,"test/phone/gyro")
test2=os.path.join(path,"test/phone/accel")
test3=os.path.join(path,"test/watch/gyro")
test4=os.path.join(path,"test/watch/accel")

In [85]:
df=pd.DataFrame()
for n in os.listdir(train1):
    df=pd.concat([df,pd.read_csv(os.path.join(train1,n),sep=",",header=None)])
for n in os.listdir(train2):
    df=pd.concat([df,pd.read_csv(os.path.join(train2,n),sep=",",header=None)])
for n in os.listdir(train3):
    df=pd.concat([df,pd.read_csv(os.path.join(train3,n),sep=",",header=None)])
for n in os.listdir(train4):
    df=pd.concat([df,pd.read_csv(os.path.join(train4,n),sep=",",header=None)])


df_test=pd.DataFrame()
for n in os.listdir(test1):
    df_test=pd.concat([df_test,pd.read_csv(os.path.join(test1,n),sep=",",header=None)])
for n in os.listdir(test2):
    df_test=pd.concat([df_test,pd.read_csv(os.path.join(test2,n),sep=",",header=None)])
for n in os.listdir(test3):
    df_test=pd.concat([df_test,pd.read_csv(os.path.join(test3,n),sep=",",header=None)])
for n in os.listdir(test4):
    df_test=pd.concat([df_test,pd.read_csv(os.path.join(test4,n),sep=",",header=None)])
df_test.head()

,0,1,2,3,4,5
0,1620,A,938035035913281,0.811447,0.440994,-1.1239166;
1,1620,A,938035086267284,0.316513,0.900757,-0.92803955;
2,1620,A,938035136621288,0.187683,0.590820,-0.17601013;
3,1620,A,938035186975292,0.340454,0.755875,0.7628479;
4,1620,A,938035237329296,0.475555,0.825500,0.10922241;


In [86]:
train=df
test=df_test
df.columns=["id","activity","time","x","y","z"]
df['z'] = df['z'].apply(lambda x: float(x[:-1]))

df_test.columns=["id","activity","time","x","y","z"]
df_test['z'] = df_test['z'].apply(lambda x: float(x[:-1]))

df['time'] = df['time'].apply(lambda x: x//1000000)
df['time'] = df['time'].apply(lambda x: datetime.fromtimestamp(x))
df_test['time'] = df_test['time'].apply(lambda x: x//1000000)
df_test['time'] = df_test['time'].apply(lambda x: datetime.fromtimestamp(x))
df.drop(columns="id",inplace=True)
df_test.drop(columns="id",inplace=True)
df_test.head()

,activity,time,x,y,z
0,A,1999-09-23 02:47:15,0.811447,0.440994,-1.123917
1,A,1999-09-23 02:48:06,0.316513,0.900757,-0.928040
2,A,1999-09-23 02:48:56,0.187683,0.590820,-0.176010
3,A,1999-09-23 02:49:46,0.340454,0.755875,0.762848
4,A,1999-09-23 02:50:37,0.475555,0.825500,0.109222


In [87]:
activity=["A","B","C","D","E","F","G","H","I","J","K","L","M","O","P","Q","R","S"]
activity_encoder={}
activity_decoder={}
count=0
for n in activity:
    activity_encoder[n]=count
    activity_decoder[count]=n
    count+=1

df['activity'] = df['activity'].apply(lambda x: activity_encoder[x])

df_test['activity'] = df_test['activity'].apply(lambda x: activity_encoder[x])
df_test.head()

,activity,time,x,y,z
0,0,1999-09-23 02:47:15,0.811447,0.440994,-1.123917
1,0,1999-09-23 02:48:06,0.316513,0.900757,-0.928040
2,0,1999-09-23 02:48:56,0.187683,0.590820,-0.176010
3,0,1999-09-23 02:49:46,0.340454,0.755875,0.762848
4,0,1999-09-23 02:50:37,0.475555,0.825500,0.109222


In [88]:
for n in ("year","month","week","day"):
    df[n]=getattr(df["time"].dt,n)
    
df.drop(columns="time",inplace=True)
for n in ("year","month","week","day"):
    df_test[n]=getattr(df_test["time"].dt,n)
df_test.drop(columns="time",inplace=True)
df_test.head()

,activity,x,y,z,year,month,week,day
0,0,0.811447,0.440994,-1.123917,1999,9,38,23
1,0,0.316513,0.900757,-0.928040,1999,9,38,23
2,0,0.187683,0.590820,-0.176010,1999,9,38,23
3,0,0.340454,0.755875,0.762848,1999,9,38,23
4,0,0.475555,0.825500,0.109222,1999,9,38,23


In [91]:
df = df.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
total_data=pd.DataFrame()
total_data=pd.concat([df,df_test])
total_data.head()

,activity,x,y,z,year,month,week,day
0,5,0.973542,-5.880608,7.619996,1972,9,37,17
1,11,0.254384,-5.629367,7.897723,1970,7,30,22
2,3,3.424606,-3.934272,8.221240,1972,7,28,14
3,8,0.329536,0.083275,0.060896,1995,6,25,22
4,9,0.004181,0.001923,0.003418,1974,1,1,1


In [92]:
y=total_data["activity"]
x=total_data.drop(columns="activity")
x_train, x_test, y_train, y_test = train_test_split(x,y , train_size = 0.8, random_state =  90)


D:\ANACONDA\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [67]:
# mm_scaler = preprocessing.Normalizer().fit(x_train)
# x_train=mm_scaler.transform(x_train)
# x_test=mm_scaler.transform(x_test)


In [93]:
model = tree.DecisionTreeClassifier()
model.fit(x_train, y_train)
y= model.predict(x_test)
acc = accuracy_score(y_test, y)
print("Accuracy of model is ",acc)

Accuracy of model is  0.9426704378314144


In [ ]:
def eval_df(df):
    df.columns=["id","activity","time","x","y","z"]
    df['z'] = df['z'].apply(lambda x: float(x[:-1]))
    df['time'] = df['time'].apply(lambda x: x//1000000)
    df['time'] = df['time'].apply(lambda x: datetime.fromtimestamp(x))
    df.drop(columns="id",inplace=True)
    activity=["A","B","C","D","E","F","G","H","I","J","K","L","M","O","P","Q","R","S"]
    activity_encoder={}
    activity_decoder={}
    count=0
    for n in activity:
        activity_encoder[n]=count
        activity_decoder[count]=n
        count+=1
    df['activity'] = df['activity'].apply(lambda x: activity_encoder[x])
    for n in ("year","month","week","day"):
        df[n]=getattr(df["time"].dt,n)
    df.drop(columns="time",inplace=True)
    return df

In [ ]:
df_for_evaluation=eval_df(your_dataframe_here)
y= model.predict(df_for_evaluation)
acc = accuracy_score(y_test, y)
print("Accuracy of model is ",acc)